# ME 574 Spring 2020  Homework #3 - Due Tuesday May 5





Some of the problems below need some example input data. For that purpose, we  construct a numpy array `v` containing the sequence of digits in $\pi$ multiplied by $0.1$.

In [51]:
import numpy as np
pi_string = "3141592653589793238462643383279502884197169399375105820974944592307816406286208998628034825342117067982148086513282306647093844609550582231725359408128481117450284102701938521105559644622948954930381964428810975665933446128475648233786783165271201909145648566923460348610454326648213393607260249141273724587006606315588174881520920962829254091715364367892590360011330530548820466521384146951941511609433057270365759591953092186117381932611793105118548074462379962749567351885752724891227938183011949129833673362440656643086021394946395224737190702179860943702770539217176293176752384674818467669405132000568127145263560827785771342757789609"
pi_digits = [int(char) for char in pi_string]
v = 0.1*np.array(digits)[0:374]

__1a)__ Using numba, write a parallel implementation of a function that computes the element-wise product of 2 input arrays based on the declaration line and doc string below:

In [ ]:
def ewprod(u,v,w):
```
Compute the element-wise product of arrays

Args:
    u,v: 1D numpy arrays

Returns:
    w: 1D numpy array containing product of corresponding entries in input arrays u and v
```
    #insert code
    return w

Your implementation should include both the wrapper function `ewprod()` and the kernel function it calls to execute the operation in parallel.

__1b)__ Use your parallel code to compute the elementwise product of `v` with `1.-v` and compare with the results from numpy's built-in `*` or `multiply()`.

__1c)__ Now for the fun part. Modify your kernel to include a statement to make any thread whose index is a multiple of 32 (`if cuda.thradIdx.x % 32 == 0`) print the block and thread index values. Set your execution parameters to create a grid with 4 blocks of 96 threads. Execute the code several times and provide terminal output illustrating you answers to the following questions:

Is the order of block execution repeatable/predictable?

Do threads with lower thread index always execute before threads in the same block with higher thread index values? (In other words, is order of execution within a block repeatable/predictable?)



__2a)__ Write a python function `smooth(v,rad)` that smooths an array of data by computing a local average; i.e. in the output each element of the input array `v[i]` that has `rad` neighbors is replaced by the average of `2*rad+1` elements (`v[i-rad],...,v[i+rad])`. Let's construct a larger input array for this problem: `w = np.outer(v,1-v).flatten()` that has $375^2$ entries.

Use your `smooth()` function to compute smoothed versions of `w`. Plot the smoothed arrays for `rad=2` and `rad=4`. Compute the python timings for each computation and include them in the legend for your plot.

__2b)__ Write numba code to create a parallelized version of `smooth()`. Verify that it reproduces your results from 2a) and make a corresponding plot (this time including timings based on cuda events).

__2c)__ Write numba code to create a "tiled" parallelization of `smooth()` that uses shared memory. Again verify that your code reproduces your results from 2a) and make a corresponding plot (including timings based on cuda events.

__3)__ Our introductory discussion of ODEs dealt with initial value problems (IVPs), but boundary value problems (BVPs), where both initial and final conditions need to be satisfied, are also relevant. An important aplication involves computation of eigenvalues and eigenfunctions. For example, consider the longitudinal vibrations of a rod fixed at one end ($x=0$) and attached to a spring with stiffness $k$ at the other end ($x=L$). The governing equations are:


$$\rho A \frac{\partial^2 u}{\partial t^2} + A E \frac{\partial^2 u}{\partial x^2}  = 0, u(0,t) = 0, \big[A E \frac{\partial u}{\partial x} - k u \big]_{x=L} = 0$$

Applying separation of variables to this problem leads to a solution in terms of normal modes:
$$u_n(x,t) =  y_n(x) sin(\omega_n t + \phi)$$

with natural frequencies (eigenvalues) and mode shapes (eigenfunctions) satisfying the following an ODE and BCs that can be written in non-dimensional form as:
$$y_n'' + \omega_n^2 y = 0, \, y(0)=0, \, y'(\pi) - c y(\pi) = 0$$

The ODE is linear, so there is an arbitrary scaling constant and we can choose the initial velocity to be 1. However, the value of the constant  $\omega$ that causes the final condition to be satisfied is not known a priori. In the serial world, a loop implementing a subdivision scheme can be used to converge to the eigenvalues. With parallel capabilities, we can launch a grid that concurrently computes numerical solutions for thousands or millions of $\omega$ values, and then use the results of those numerical solutions to identify the eigenvalues. This question leads you through implementing such an approach:

__3a)__ Write python code to implement the $4^{th}$-order Runge Kutta ODE solver, and use it to compute the numerical solution of 
$$y'' + y = 0, \, y(0) = 0, y'(0) = 1$$
(Remember to convert to a $1^{st}$-order system.)

What is the global error at $x=\pi$ if the number of steps in the solution is 10? 100?

__3b)__ Launch a 1D computational grid to compute solutions for
$$y'' + \omega^2 y = 0, \, y(0) = 0, y'(0) = 1$$
for values of $\omega \in [0,10]$ and return an array of values for $y'(\pi) - c y(\pi)$ with $c=1$. Based on finding sign changes in the array of results, determine the eigenvalues in the interval of interest.

__3c)__ Use a `for()` loop and your code from 2b) to produce a plot of eigenvalues as a function of $c$ for $c \in [0,5]$. Briefly describe/discuss the results shown in your plot. Do your results coincide with known analytic results in the limits of very small and very large values of $c$?

__OPTIONAL: 3d)__ Write numba code that produces results equivalent to those obtained in 3c) but uses a fully parallel approach with a 2D computational grid instead of looping over $c$ values.

__4)__ Write a python implementation of a Jacobi iteration solver for the 2D Poisson equation that provides a model for numerous engineering properties  including the steady-state heat distribution in a plate. Assume that your system has a single source that imposes $u(0,0) = 1$ and $u(x,y) = 0.5$ for $x>0$ and $u(x,y) = -1$ for $x<0$ in the exterior region, which is described implicitly by $f(x,y) > 0$.

a) Compute the solution for a region with the following defining function $ f_0(x,y) = x^4 + y^4 -1$, and provide a 3D surface plot illustrating your results.

b) Determine the execution time for 100 iterations on a $128 \times 128$ grid.



__5)__ Parallelize your solution to Problem 4. 

a) Verify that your parallel implementation reproduces your serial results.

b) Determine the execution time for 100 iterations on a $128 \times 128$ grid, and compute the acceleration factor due to parallelization.
